In [4]:
import os
import numpy as np
import rasterio
from PIL import Image
import os

def slice_image(image, tile_size=512, overlap=0, id=None):
    # === CONFIG ===
    band_paths = {
        "B02": image + "_B02_(Raw).tiff",  # Blue
        "B03": image + "_B03_(Raw).tiff",  # Green
        "B04": image + "_B04_(Raw).tiff",  # Red
        "B08": image + "_B08_(Raw).tiff",  # NIR
    }
    output_dir = "../data/tiles"
    tile_size = 512
    overlap = 0

    os.makedirs(output_dir, exist_ok=True)

    # === LOAD ALL BANDS ===
    bands = {}
    shape = None

    for band, path in band_paths.items():
        with rasterio.open(path) as src:
            data = src.read(1)
            bands[band] = data
            if shape is None:
                shape = data.shape

    height, width = shape

    # === TILE SLICING ===
    tile_id = 0
    for y in range(0, height - tile_size + 1, tile_size - overlap):
        for x in range(0, width - tile_size + 1, tile_size - overlap):
            # Extract patches
            patch = {
                band: bands[band][y:y+tile_size, x:x+tile_size]
                for band in band_paths
            }

            # True color RGB: B04 (R), B03 (G), B02 (B)
            r = patch["B04"]
            g = patch["B03"]
            b = patch["B02"]
            rgb = np.stack([r, g, b], axis=-1)

            # Normalize to 0-255
            rgb_norm = ((rgb - np.min(rgb)) / (np.ptp(rgb) + 1e-5) * 255).astype(np.uint8)

            im_rgb = Image.fromarray(rgb_norm)
            im_rgb.save(os.path.join(output_dir, f"tile_{tile_id}_{id}_rgb.png"))

            # False color (optional): B08 (NIR), B04 (R), B03 (G)
            nir = patch["B08"]
            fc = np.stack([nir, r, g], axis=-1)
            fc_norm = ((fc - np.min(fc)) / (np.ptp(fc) + 1e-5) * 255).astype(np.uint8)

            im_fc = Image.fromarray(fc_norm)
            im_fc.save(os.path.join(output_dir, f"tile_{tile_id}_{id}_false.png"))

            tile_id += 1


In [5]:
slice_image("../data/raw_images/01.02.2025 - suez canal/2025-02-01-00_00_2025-02-01-23_59_Sentinel-2_L2A", id='suez_01.02.25')
slice_image("../data/raw_images/09.04.2025 - malacca strait/2025-04-09-00_00_2025-04-09-23_59_Sentinel-2_L2A", id='malacca_09.04.25')
slice_image("../data/raw_images/11.04.2025 - cape of good hope/2025-04-11-00_00_2025-04-11-23_59_Sentinel-2_L2A", id='cape_11.04.25')
slice_image("../data/raw_images/14.04.2025 - suez canal/2025-04-14-00_00_2025-04-14-23_59_Sentinel-2_L2A", id='suez_14.04.25')
slice_image("../data/raw_images/19.04.2025 - malacca strait/2025-04-19-00_00_2025-04-19-23_59_Sentinel-2_L2A", id='malacca_19.04.25')
slice_image("../data/raw_images/18.07.2024 - north sea/2024-07-18-00_00_2024-07-18-23_59_Sentinel-2_L2A", id='north_sea_18.07.24')